In [0]:
#                              TRIPET NET



# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
import numpy as np
from google.colab import drive
import sys
import os
from tensorflow.keras import backend as K


drive.mount('/content/gdrive', force_remount=True)
project_path = "/content/gdrive/My Drive/shared/Colab Notebooks/tesi/models"           #PATH NEED TO BE CHANGED ACCORDING TO THE LOCATION OF THE PROJECT
data_path = "/content/gdrive/My Drive/shared/Colab Notebooks/tesi/data/"
weights_path = project_path + '/weights/'
sys.path.append(project_path)

from evaluation_utilities import *
from data_utilities import *
from net_utilities import *
from generators_utilities import *


#*******************************************************************************

In [0]:
#************************************PARAMS*************************************


n_classes = 169
batch_size = 128  # 32
random_seed = 1995
n_epoch = 1999
input_size = (28, 28, 1)

np.random.seed(seed=random_seed)
tf.random.set_seed(seed=random_seed)

#*******************************************************************************

In [0]:

def _pairwise_distances(embeddings, squared=False):
    """Compute the 2D matrix of distances between all the embeddings.
    Args:
        embeddings: tensor of shape (batch_size, embed_dim)
        squared: Boolean. If true, output is the pairwise squared euclidean distance matrix.
                 If false, output is the pairwise euclidean distance matrix.
    Returns:
        pairwise_distances: tensor of shape (batch_size, batch_size)
    """
    # Get the dot product between all embeddings
    # shape (batch_size, batch_size)
    dot_product = tf.linalg.matmul(embeddings, tf.transpose(embeddings))

    # Get squared L2 norm for each embedding. We can just take the diagonal of `dot_product`.
    # This also provides more numerical stability (the diagonal of the result will be exactly 0).
    # shape (batch_size,)
    square_norm = tf.linalg.diag_part(dot_product)

    # Compute the pairwise distance matrix as we have:
    # ||a - b||^2 = ||a||^2  - 2 <a, b> + ||b||^2
    # shape (batch_size, batch_size)
    distances = tf.expand_dims(square_norm, 1) - 2.0 * dot_product + tf.expand_dims(square_norm, 0)

    # Because of computation errors, some distances might be negative so we put everything >= 0.0
    distances = tf.math.maximum(distances, 0.0)

    if not squared:
        # Because the gradient of sqrt is infinite when distances == 0.0 (ex: on the diagonal)
        # we need to add a small epsilon where distances == 0.0
        mask = tf.compat.v1.to_float(tf.equal(distances, 0.0))
        distances = distances + mask * 1e-16

        distances = tf.math.sqrt(distances)

        # Correct the epsilon added: set the distances on the mask to be exactly 0.0
        distances = distances * (1.0 - mask)

    return distances


def _get_anchor_negative_triplet_mask(labels):
    """Return a 2D mask where mask[a, n] is True iff a and n have distinct labels.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check if labels[i] != labels[k]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))

    mask = tf.math.logical_not(labels_equal)

    return mask


def _get_anchor_positive_triplet_mask(labels):
    """Return a 2D mask where mask[a, p] is True iff a and p are distinct and have same label.
    Args:
        labels: tf.int32 `Tensor` with shape [batch_size]
    Returns:
        mask: tf.bool `Tensor` with shape [batch_size, batch_size]
    """
    # Check that i and j are distinct
    indices_equal = tf.dtypes.cast(tf.eye(tf.shape(labels)[0]), tf.bool)
    indices_not_equal = tf.math.logical_not(indices_equal)

    # Check if labels[i] == labels[j]
    # Uses broadcasting where the 1st argument has shape (1, batch_size) and the 2nd (batch_size, 1)
    labels_equal = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    
    # Combine the two masks
    mask = tf.math.logical_and(indices_not_equal, labels_equal)

    return mask

def _get_anchor_positive_triplet_mask_multi_domain(labels):

    # *************************multi domain version*****************************
    mask = tf.math.equal(tf.expand_dims(labels, 0), tf.expand_dims(labels, 1))
    # **************************************************************************

    return mask

    # *************************multi domain version*****************************
def multidomain_pairwise_dist(A, B):
    """
    Computes pairwise distances between each elements of A and each elements of B.
    Args:
      A,    [m,d] matrix
      B,    [n,d] matrix
    Returns:
      D,    [m,n] matrix of pairwise distances
    """
    #with tf.variable_scope('pairwise_dist'):
    # squared norms of each row in A and B
    na = tf.reduce_sum(tf.square(A), 1)
    nb = tf.reduce_sum(tf.square(B), 1)

    # na as a row and nb as a co"lumn vectors
    na = tf.reshape(na, [-1, 1])
    nb = tf.reshape(nb, [1, -1])

    # return pairwise euclidean difference matrix
    D = tf.sqrt(tf.maximum(na - 2 * tf.matmul(A, B, False, True) + nb, 0.0))
    return D
    # **************************************************************************


'''
    - INPUT:
        - y_pred = [emb1_d1, emb2_d1, emb3_d1, ..., emb1_d2, emb2_d2, emb3_d2, ...]
        - y_true = [label1_d1, label2_d1, label3_d1, ..., label1_d2, label2_d2, label3_d3, ...] (labels must be the same in the 2 domains)

'''
def batch_hard_triplet_loss_multi_domain(y_true, y_pred, margin=1, squared=False):

    #*************************keras need this***********************************
    labels = tf.squeeze(y_true, axis=-1)
    #***************************************************************************
  
    # *************************multi domain version*****************************
    #labels = labels[:labels.shape[0] // 2]                                      # labels must be the same in the 2 domains
    _batch_size = y_pred.shape[1] // 2

    # embedding = (batch, feat)
    embedding_d = y_pred[:, :_batch_size]
    embedding_i = y_pred[:, _batch_size:]
    
    pairwise_dist_same = multidomain_pairwise_dist(embedding_i, embedding_i)
    pairwise_dist_diff = multidomain_pairwise_dist(embedding_i, embedding_d)

    pairwise_dist = tf.linalg.set_diag(
        pairwise_dist_diff,
        tf.linalg.diag_part(                                                    # all zeros
            pairwise_dist_same,
        )
    )

    
    # **************************************************************************
    # *************************single domain version****************************
    #pairwise_dist = _pairwise_distances(y_pred, squared=squared)
    # **************************************************************************

    # For each anchor, get the hardest positive
    # First, we need to get a mask for every valid positive (they should have same label)
    
    # *************************multi domain version*****************************
    mask_anchor_positive = _get_anchor_positive_triplet_mask_multi_domain(labels)
    # **************************************************************************
    # *************************single domain version*****************************
    #mask_anchor_positive = _get_anchor_positive_triplet_mask(labels)
    # **************************************************************************

    mask_anchor_positive = tf.compat.v1.to_float(mask_anchor_positive)

    # We put to 0 any element where (a, p) is not valid (valid if a != p and label(a) == label(p))
    anchor_positive_dist = tf.math.multiply(mask_anchor_positive, pairwise_dist)

    # shape (batch_size, 1)
    hardest_positive_dist = tf.math.reduce_max(anchor_positive_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_positive_dist", tf.math.reduce_mean(hardest_positive_dist))

    # For each anchor, get the hardest negative
    # First, we need to get a mask for every valid negative (they should have different labels)
    mask_anchor_negative = _get_anchor_negative_triplet_mask(labels)
    mask_anchor_negative = tf.compat.v1.to_float(mask_anchor_negative)

    # We add the maximum value in each row to the invalid negatives (label(a) == label(n))
    max_anchor_negative_dist = tf.math.reduce_max(pairwise_dist, axis=1, keepdims=True)
    anchor_negative_dist = pairwise_dist + max_anchor_negative_dist * (1.0 - mask_anchor_negative)

    # shape (batch_size,)
    hardest_negative_dist = tf.math.reduce_min(anchor_negative_dist, axis=1, keepdims=True)
    tf.summary.scalar("hardest_negative_dist", tf.math.reduce_mean(hardest_negative_dist))

    # Combine biggest d(a, p) and smallest d(a, n) into final triplet loss
    triplet_loss = tf.math.maximum(hardest_positive_dist - hardest_negative_dist + margin, 0.0)

    # Get final mean triplet loss
    triplet_loss = tf.math.reduce_mean(triplet_loss)

    return triplet_loss

#*********************************************************************************************************************************************

In [0]:
#******************************DATA PROCESSING**********************************


print("loading data...")


# **** load draws ****
(X_draws, y_string_draws) = load_data(data_path + "/draws-28.pickle", size=input_size[0], _3d=False, invert=False, randomize=False, rand_seed=random_seed)

# **** load icons ****
(X_icons, y_string_icons) = load_data(data_path + "/icons-28.pickle", size=input_size[0], _3d=False, invert=False, randomize=False, rand_seed=random_seed)


# **** check datasets classes ****
X_draws, X_icons, y_string_draws, y_string_icons = check_dataset_classes(X_draws, X_icons, y_string_draws, y_string_icons)


# **** preprocess  draws ****
x_train_draws, x_valid_draws, x_test_draws, y_train_draws, y_valid_draws, y_test_draws = split_dataset(X_draws, y_string_draws, _validation_size=0.2, _test_size=0.1, _random_seed=random_seed)
y_train_draws, y_valid_draws, y_test_draws = labels_preprocessing(y_train_draws, y_valid_draws, y_test_draws)
x_train_draws, y_train_draws = shuffle_with_same_indexes(x_train_draws, y_train_draws, seed=random_seed)
x_valid_draws, y_valid_draws = shuffle_with_same_indexes(x_valid_draws, y_valid_draws, seed=random_seed)
x_test_draws, y_test_draws = shuffle_with_same_indexes(x_test_draws, y_test_draws, seed=random_seed)
x_train_draws, x_valid_draws, x_test_draws = data_preprocessing(x_train_draws), data_preprocessing(x_valid_draws), data_preprocessing(x_test_draws)



# **** preprocess  icons ****
x_train_icons, x_valid_icons, x_test_icons, y_train_icons, y_valid_icons, y_test_icons = split_dataset(X_icons, y_string_icons, _validation_size=0.2, _test_size=0.1, _random_seed=random_seed)
y_train_icons, y_valid_icons, y_test_icons = labels_preprocessing(y_train_icons, y_valid_icons, y_test_icons)
x_train_icons, y_train_icons = shuffle_with_same_indexes(x_train_icons, y_train_icons, seed=random_seed)
x_valid_icons, y_valid_icons = shuffle_with_same_indexes(x_valid_icons, y_valid_icons, seed=random_seed)
x_test_icons, y_test_icons = shuffle_with_same_indexes(x_test_icons, y_test_icons, seed=random_seed)
x_train_icons, x_valid_icons, x_test_icons = data_preprocessing(x_train_icons), data_preprocessing(x_valid_icons), data_preprocessing(x_test_icons)



# **** check datasets ****
X_draws, X_icons, y_string_draws, y_string_icons = check_dataset_classes(X_draws, X_icons, y_string_draws, y_string_icons)
assert len(set(y_train_draws)) == n_classes, print("wrong class number")
assert len(set(y_train_icons)) == n_classes, print("wrong class number")

print("data loaded")

#*******************************************************************************

In [0]:
#***********************************NETWORK*************************************


class SingleNet(tf.keras.Model):

  def __init__(self):
    super(SingleNet, self).__init__()
    self.filter_size = 24

    self.l1_conv    = tf.keras.layers.Conv2D(self.filter_size,kernel_size=3,activation='relu',input_shape=(28,28,1), name="l1_conv")
    self.l1_batch   = tf.keras.layers.BatchNormalization(name="l1_batch")
    self.l2_conv    = tf.keras.layers.Conv2D(self.filter_size,kernel_size=3,activation='relu', name="l2_conv")
    self.l2_batch   = tf.keras.layers.BatchNormalization(name="l2_batch")
    self.l3_conv    = tf.keras.layers.Conv2D(self.filter_size,kernel_size=5,strides=2,padding='same',activation='relu', name="l3_conv")
    self.l3_batch   = tf.keras.layers.BatchNormalization(name="l3_batch")
    self.l3_dropout = tf.keras.layers.Dropout(0.4, name="l3_dropout")
    self.l4_conv    = tf.keras.layers.Conv2D(self.filter_size*2,kernel_size=3,activation='relu', name="l4_conv")
    self.l4_batch   = tf.keras.layers.BatchNormalization(name="l4_batch")

  
  def call(self, x):
    x = self.l1_conv(x)
    x = self.l1_batch(x)
    x = self.l2_conv(x)
    x = self.l2_batch(x)
    x = self.l3_conv(x)
    x = self.l3_batch(x)
    x = self.l3_dropout(x)
    x = self.l4_conv(x)
    x = self.l4_batch(x)

    return x


class SharedNet(tf.keras.Model):

  def __init__(self, n_classes=None):
    super(SharedNet, self).__init__()

    self.n_classes = n_classes
    self.filter_size = 24

    self.l5_conv    = tf.keras.layers.Conv2D(self.filter_size*2,kernel_size=3,activation='relu', name="l5_conv")
    self.l5_batch   = tf.keras.layers.BatchNormalization(name="l5_batch")
    self.l6_conv    = tf.keras.layers.Conv2D(self.filter_size*2,kernel_size=5,strides=2,padding='same',activation='relu', name="l6_conv")
    self.l6_batch   = tf.keras.layers.BatchNormalization(name="l6_batch")
    self.l6_dropout = tf.keras.layers.Dropout(0.4, name="l6_dropout")
    self.l7_flatten = tf.keras.layers.Flatten(name="l7_flatten")
    self.l7_dense   = tf.keras.layers.Dense(128, activation='relu', name="l7_dense")

    if self.n_classes:
      self.l8_batch   = tf.keras.layers.BatchNormalization(name="classification1")
      self.l8_dropout = tf.keras.layers.Dropout(0.4, name="classification2")
      self.l8_dense   = tf.keras.layers.Dense(n_classes, activation='softmax', name="classification3")
  
  def call(self, x):
    x = self.l5_conv(x)
    x = self.l5_batch(x)
    x = self.l6_conv(x)
    x = self.l6_batch(x)
    x = self.l6_dropout(x)
    x = self.l7_flatten(x)
    x = self.l7_dense(x)

    if self.n_classes:
      x = self.l8_batch(x)
      x = self.l8_dropout(x)
      x = self.l8_dense(x)
    
    return x

class BranchNet(tf.keras.Model):

  def __init__(self, shared_net=None, n_classes=None):
    super(BranchNet, self).__init__()

    self.single_net = SingleNet()

    # if a sharedNet instance is passed in args, then this net uses the reference
    # of the passed net, otherwise it creates a new BranchNet instance
    if shared_net is None:   
      self.shared_net = SharedNet(n_classes) if n_classes else SharedNet()   
    else:
      if n_classes: print("n_classes parameter ignored")
      self.shared_net = shared_net


    self.single_net._name = "single_net"
    self.shared_net._name = "shared_net"
      

  def call(self, image):
    image = self.single_net(image)
    image = self.shared_net(image)
    return image




class TripletNet(tf.keras.Model):

  def __init__(self):
    super(TripletNet, self).__init__()

    self.shared_net = SharedNet()

    self.draws_net = BranchNet(self.shared_net)
    self.icons_net = BranchNet(self.shared_net)


  def call(self, img_input):

    draw, icon = img_input

    draw = self.draws_net.call(draw)
    icon = self.icons_net.call(icon)

    return tf.keras.layers.concatenate([draw, icon])


lr = 0.0001

drawsIconsTriplet = TripletNet()
drawsIconsTriplet.compile(tf.keras.optimizers.Adam(lr), loss=batch_hard_triplet_loss_multi_domain, run_eagerly=True) 
  
drawsIconsTriplet.build([(1, 28, 28,1), (1, 28, 28,1)])
_ = drawsIconsTriplet.call(
    [tf.zeros((1, 28, 28, 1)), tf.zeros((1, 28, 28, 1))]
)
#*******************************************************************************


In [0]:
tl_net = SimpleNet(n_classes=345)
tl_net.compile(tf.keras.optimizers.Adam(0.0001), loss=tf.keras.losses.sparse_categorical_crossentropy, metrics=['acc'])
tl_net.build((1, 28, 28,1))


tl_net.load_weights(weights_path + "/cnn_quickdraw15_filter24.h5")
tl_net_embedding = tl_net.embedding_net

load_nested_net_weights(tl_net_embedding, drawsIconsTriplet.draws_net)
load_nested_net_weights(tl_net_embedding, drawsIconsTriplet.icons_net)


In [0]:

# **** create generators ****

is_data_already_prepro = False if np.max(x_train_draws[0]) > 1 else True
print("is data preprocessed? ", is_data_already_prepro)

custom_aug = CustomAug({
      'rescale': not is_data_already_prepro,        # if rescale == True, the alg assumes the data is in format 0-255
      'pad' : True,                           
      'horizontal_flip' : True,             
      'erosion' : True,                       
      'half_aug' : True,
    }
)

augment_draws = False
if augment_draws:
  triplet_training_generator = TripletDataGenerator(x_train_draws, y_train_draws, x_train_icons, y_train_icons, batch_size=batch_size, shuffle=True, transf_anchor_func=custom_aug.custom_preprocessing, transf_image_func=None) 
  triplet_validation_generator = TripletDataGenerator(x_valid_draws, y_valid_draws, x_valid_icons, y_valid_icons, batch_size=batch_size, shuffle=True, transf_anchor_func=custom_aug.custom_preprocessing, transf_image_func=None)
else:
  triplet_training_generator = TripletDataGenerator(x_train_draws, y_train_draws, x_train_icons, y_train_icons, batch_size=batch_size, shuffle=True) 
  triplet_validation_generator = TripletDataGenerator(x_valid_draws, y_valid_draws, x_valid_icons, y_valid_icons, batch_size=batch_size, shuffle=True)



is data preprocessed?  True


In [0]:
#**********************************TRAINING*************************************

net_callbacks = [
	PlotLosses(),
  tf.keras.callbacks.ModelCheckpoint(weights_path + 'triplet_' + "{epoch:02d}"  + '.h5',  monitor='val_loss', verbose=1, period=5, save_best_only=True, mode='min'),
	#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, min_delta=1, verbose=1, mode='auto', restore_best_weights=True)
]
print("net_callbacks var created")


drawsIconsTriplet.fit(
    triplet_training_generator,
    epochs=n_epoch,
    verbose=2, 
    callbacks=net_callbacks,
    validation_data=triplet_validation_generator,
)




#*******************************************************************************

In [0]:
drawsIconsTriplet.load_weights(weights_path + 'no_aug_triplet_20.h5')

In [0]:
X_icons_eval = load_data(path=data_path + "/icons_eval.pickle", size=input_size[0], invert=False, _3d=False, randomize=False, rand_seed=random_seed)

# delete white images
X_icons_eval = np.asarray([i for i in X_icons_eval if np.min(i) != np.max(i)])

X_icons_eval_edges = data_preprocessing(X_icons_eval)

#for _i, i in enumerate(X_icons_eval_edges): X_icons_eval_edges[_i] = contour_img(i)

for i, im in enumerate(X_icons_eval_edges):
  if i < 10:
    print(np.min(im), np.max(im))
    show_img(im)

In [0]:
print("triplet network trained:")

manual_eval = RankImages(X_icons_eval_edges, drawsIconsTriplet.icons_net, drawsIconsTriplet.draws_net, _n=15, _show_im=False, _show_dist=False)
#target_im = paint_brush_img(w=input_size[0], h=input_size[1], line_width=30, preprocessed=True, show=False)
images = [cv2.imread(os.path.join(data_path, 'targets', im_path), 0) for im_path in os.listdir(os.path.join(data_path, 'targets')) if im_path.endswith('.jpg')]
images = [cv2.resize(i, (input_size[0], input_size[1])) for i in images]

for image in images:
  target_im = image
  target_im = np.expand_dims(data_preprocessing(target_im), axis=-1)
  #target_im = contour_img(target_im)
  res = manual_eval.get_n_most_similar_images(target_im, _returnType='indexes')
  manual_eval.format_result(target_im, [X_icons_eval[r] for r in res])

In [0]:
print("creating vector space...")
x_feat_test_icons = np.array([drawsIconsTriplet.icons_net.predict(f[np.newaxis, ...]) for f in x_test_icons])[:, 0, :]
x_feat_test_draws = np.array([drawsIconsTriplet.draws_net.predict(f[np.newaxis, ...]) for f in x_test_draws])[:, 0, :]

print("vector space created")

knn = KNN(x_feat_test_icons, y_test_icons, k=10)

y_test_icons_true = y_test_icons                                                # np.array_equal(y_test_icons,y_test_icons) is True
y_test_draws_pred = knn.get_labels(x_feat_test_draws)

get_score(y_test_icons_true, y_test_draws_pred)



